In [1]:
USE {
    repositories {
        mavenCentral()
        maven(url = "https://repo.spring.io/milestone")
        maven(url = "https://repo.spring.io/snapshot")
    }

    dependencies {
        val springAiVersion = "1.0.0-M5"
        implementation("org.springframework.ai:spring-ai-openai:$springAiVersion")
        implementation("org.springframework.ai:spring-ai-openai-spring-boot-starter:$springAiVersion")
        implementation("com.fasterxml.jackson.module:jackson-module-kotlin:2.18.2")
    }
}

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

In [3]:
import org.springframework.ai.openai.OpenAiChatModel
import org.springframework.ai.openai.OpenAiChatOptions
import org.springframework.ai.openai.api.OpenAiApi

val openAiApi = OpenAiApi(apiKey)
val openAiChatOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_4_O_MINI)
    .temperature(0.3)
    .build()
val chatModel = OpenAiChatModel(openAiApi, openAiChatOptions)

## Chat API

In [4]:
chatModel.call("How do I create in Kotlin a plus function for a Point data class that adds two Points together in the form point1 + point2?")

In Kotlin, you can overload the `plus` operator for a data class by defining the `plus` function. To create a `Point` data class that supports adding two `Point` instances using the `+` operator, you can follow these steps:

1. Define the `Point` data class with properties for the x and y coordinates.
2. Override the `plus` operator by defining a function named `plus` that takes another `Point` as a parameter and returns a new `Point` instance with the summed coordinates.

Here's an example implementation:

```kotlin
data class Point(val x: Int, val y: Int) {
    operator fun plus(other: Point): Point {
        return Point(this.x + other.x, this.y + other.y)
    }
}

fun main() {
    val point1 = Point(1, 2)
    val point2 = Point(3, 4)

    val result = point1 + point2 // Using the overloaded plus operator
    println(result) // Output: Point(x=4, y=6)
}
```

### Explanation:
- The `Point` data class has two properties: `x` and `y`, both of type `Int`.
- The `plus` function is marked

## Prompts

In [5]:
import org.springframework.ai.chat.client.ChatClient

val chatClient = ChatClient.builder(chatModel).defaultSystem(
    """
    You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
    - Use clean code principles.
    - Write concise and readable APIs.
    - Stick to idiomatic Kotlin conventions.
    - Leverage higher-order functions, extensions, and null-safety features.
    - Avoid unnecessary complexity; simplify the code when possible.
    - Ensure the code is optimized for performance.
    - Structure the code clearly and use best practices.
    - Provide concise and to-the-point responses.
    """.trimIndent()
).build()

In [6]:
chatClient
    .prompt()
    .user {
        it.text("""
        What foldRight does? What will be the intermidieate and the end result of the foldRight function call?
        ```kotlin
            "baba".foldRight("") { x, acc -> "${'$'}x${'$'}{acc}${'$'}{acc}" })
        ```
        """.trimIndent())
    }
    .call()
    .content()


In Kotlin, the `foldRight` function is used to accumulate a value from the right side of a collection. It takes an initial value and a lambda function that combines each element with an accumulator, starting from the last element of the collection and moving to the first.

Here's a breakdown of the `foldRight` function call you provided:

```kotlin
"baba".foldRight("") { x, acc -> "$x${acc}${acc}" }
```

### Explanation:
- The string `"baba"` is treated as a collection of characters: `['b', 'a', 'b', 'a']`.
- The initial accumulator value is an empty string `""`.
- The lambda function takes two parameters:
  - `x`: the current character from the string.
  - `acc`: the accumulated result up to that point.

### Intermediate Steps:
1. Start with the last character `'a'`:
   - `x = 'a'`, `acc = ""` → Result: `"a"`
   
2. Move to the second last character `'b'`:
   - `x = 'b'`, `acc = "a"` → Result: `"bab"`
   
3. Move to the third last character `'a'`:
   - `x = 'a'`, `acc = "bab"` → Resul

or

In [7]:
val chatClient = ChatClient.builder(chatModel).build()

chatClient
    .prompt()
    .system {
        it.text(
            """
            You are an expert Kotlin developer. Your task is to write idiomatic Kotlin code. Follow these guidelines:
            - Use clean code principles.
            - Write concise and readable APIs.
            - Stick to idiomatic Kotlin conventions.
            - Leverage higher-order functions, extensions, and null-safety features.
            - Avoid unnecessary complexity; simplify the code when possible.
            - Ensure the code is optimized for performance.
            - Structure the code clearly and use best practices.
            - Provide concise and to-the-point responses.
            """.trimIndent()
        )
    }
    .user {
        it.text(
            """
        What foldRight does? What will be the intermidieate and the end result of the foldRight function call?
        ```kotlin
            "baba".foldRight("") { x, acc -> "${'$'}x${'$'}{acc}${'$'}{acc}" })
        ```
        """.trimIndent()
        )
    }
    .call()
    .content()


The `foldRight` function in Kotlin is a higher-order function that processes a collection (or a string, in this case) from right to left, accumulating a result. It takes an initial value (the accumulator) and a lambda function that defines how to combine each element with the accumulator.

In your example, the string `"baba"` is being processed with an initial accumulator of `""` (an empty string). The lambda function takes two parameters: `x`, which represents the current character being processed, and `acc`, which is the accumulated result up to that point.

Here's a breakdown of the intermediate steps and the final result:

1. **Initial Call**: `foldRight` starts with the last character of the string and the initial accumulator.
2. **Processing Steps**:
   - For the last character `'a'`: 
     - `x = 'a'`, `acc = ""` → Result: `"a"`
   - For the second last character `'b'`: 
     - `x = 'b'`, `acc = "a"` → Result: `"bab"`
   - For the third character `'a'`: 
     - `x = 'a'`, `acc =

## Stream

In [8]:
val response = chatModel.stream(
    """
    Write a main function that creates a flow of five elements 1, 2, 3, 4, and 5, and prints each one of them. Use the collect function without any arguments to do so.
    Make sure to include all imports, but don’t use star imports.
    """.trimIndent()
)

response
    .doOnNext(::println)
    .blockLast()

## Structured Output

### JSON_MODE

In [9]:
import com.fasterxml.jackson.annotation.JsonCreator
import org.springframework.ai.openai.api.ResponseFormat

/**
 * Represents a person with basic personal attributes.
 *
 * @property name The name of the person.
 * @property age The age of the person in years.
 * @property height The height of the person in meters.
 * @property married Indicates if the person is married.
 */

data class Person(val name: String, val age: Int, val height: Double, val married: Boolean)

val openAiChatOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_4_O_MINI)
    .temperature(0.3)
    .responseFormat(ResponseFormat.builder().type(ResponseFormat.Type.JSON_OBJECT).build())
    .build()
val chatModel = OpenAiChatModel(openAiApi, openAiChatOptions)

In [10]:
ChatClient.create(chatModel)
    .prompt()
    .user(
        """
        John is 42 years old and lives an independent life.
        He stands 1.75 meters tall and carries himself with confidence.
        Currently unmarried, he enjoys the freedom to focus on his personal goals and interests.
        """
    )
    .call()
    .entity(Person::class.java)

Person(name=John, age=42, height=1.75, married=false)

## Function calling

In [11]:
import org.springframework.context.annotation.Bean
import org.springframework.context.annotation.Configuration
import org.springframework.context.annotation.Description
import java.util.function.Function

class Tools {
    fun plus(a: Int, b: Int): Int = a + b
    fun minus(a: Int, b: Int): Int = a - b
    fun times(a: Int, b: Int): Int = a * b
    fun div(a: Int, b: Int): Int = a / b
}

data class MathRequest(
    val operation: String,
    val a: Int,
    val b: Int
)

data class MathResponse(val result: Int)



 @Bean
 @Description("Math operations such as addition, subtraction, multiplication, and division.")
 fun mathFunction(): Function<MathRequest, MathResponse> = MathService()


class MathService: Function<MathRequest, MathResponse> {
    private val tools = Tools()

    override fun apply(request: MathRequest): MathResponse {
        val result = when (request.operation) {
            "plus" -> tools.plus(request.a, request.b)
            "minus" -> tools.minus(request.a, request.b)
            "times" -> tools.times(request.a, request.b)
            "div" -> tools.div(request.a, request.b)
            else -> throw IllegalArgumentException("Unknown operation: ${request.operation}")
        }
        return MathResponse(result)
    }
}

val openAiChatOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_4_O_MINI)
    .temperature(0.3)
    .build()
val chatModel = OpenAiChatModel(openAiApi, openAiChatOptions)
val chatClient = ChatClient.create(chatModel)


In [12]:
import org.springframework.ai.model.function.FunctionCallback

val response = chatClient.prompt()
    .system("You are a helpful math tutor.")
    .user("""
Solve this:
* (5 + 10) * (20 - 3) / 4
* (a + b)^2
* ax^2 + bx + c = 0 for a=1, b=-3, c=-4
* S = n * (a1 + an) / 2
    n = 10
    a1 = 3
    an = 20
""")
    .function("mathFunction", "Math operations such as plus, minus, times, and div.", MathService())
    .call()
    .content()

println(response)

Here are the solutions to your problems:

1. **(5 + 10) * (20 - 3) / 4**:
   - \( (15) * (17) / 4 = 255 / 4 = 63.75 \)

2. **(a + b)^2**: This expression is not evaluated because values for \(a\) and \(b\) were not provided.

3. **Solve \( ax^2 + bx + c = 0 \) for \( a=1, b=-3, c=-4 \)**:
   - The roots of the equation can be calculated using the quadratic formula \( x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a} \).
   - Here, \( b^2 - 4ac = (-3)^2 - 4(1)(-4) = 9 + 16 = 25 \).
   - Thus, the roots are \( x = \frac{3 \pm 5}{2} \), which gives \( x = 4 \) and \( x = -1 \).

4. **S = n * (a1 + an) / 2** with \( n = 10, a1 = 3, an = 20 \):
   - \( S = 10 * (3 + 20) / 2 = 10 * 23 / 2 = 115 \).

In summary:
- The result of the first expression is **63.75**.
- The roots of the quadratic equation are **4 and -1**.
- The sum \( S \) is **115**. 

If you have values for \( a \) and \( b \) in the second expression, please provide them for evaluation!
